In [ ]:
import descarteslabs as dl
from descarteslabs.vector import Feature, Table, TableOptions, models
from pydantic import Field

In [ ]:
import requests
import geopandas as gpd

In [ ]:
org = dl.auth.Auth().payload["org"]

In [ ]:
gdf = gpd.read_parquet("data/countries.geoparquet")
gdf.plot(figsize=(10, 5))

In [ ]:
gdf.head(5)

In [ ]:
gdf.info()

## Table Models

* Subclasses the pydantic model
* geometry by default

In [ ]:
class CountryModel(models.MultiPolygonBaseModel):
    NAME: str = Field(json_schema_extra={"index": True})
    REGION_UN: str
    CONTINENT: str
    POP_EST: float
    LASTCENSUS: float

In [ ]:
# create the table with the default generic feature model
the_world = Table.create(
    "the-world", name="The World", owners=["org:descarteslabs"], model=CountryModel
)

In [ ]:
borders = the_world.add(gdf)

In [ ]:
print(the_world.description)
the_world.description = "Country boundaries for the world."
the_world.save()
print(the_world.description)

In [ ]:
# add readers to the product
print(the_world.readers)
the_world.readers.append("org:pga-tour")
the_world.save()
print(the_world.readers)

In [ ]:
# these are equivalent access patterns
feat1 = Feature.get(f"{the_world.id}:{borders.iloc[0].uuid}")
feat2 = the_world.get_feature(borders.iloc[0].uuid)
assert feat1.values == feat2.values

In [ ]:
# modify the geometry
feat1.values["geometry"] = feat1.values["geometry"].convex_hull

In [ ]:
# modify the population column
feat1.values["POP_EST"] += 10000

In [ ]:
# save the feature
feat1.save()

In [ ]:
# retrieve the Feature once again to verify the modifications
feat1 = Feature.get(f"{the_world.id}:{borders.iloc[0].uuid}")
feat1.values

In [ ]:
table = Table.get(f"{org}:the-world")
table.delete()